# MoDS and OTEAPI-tutorial

> **NOTE**: For this tutorial, we assume that you started the `docker-compose.yml`-file in the top-level directory of this repository. 
>
> Before doing so, you will also need to specify some environmental variables in this case. Make a copy of the `.env.template` with:
>
> ```
> cp .env.template .env
> ```
>
> Please generate an access token for Gitlab with the  `read_api` scope and put it under the following variable: 
>
> `GITLAB_ACCESS_TOKEN=<your-token>`
>
>
> Login into the docker registry of the Fraunhofer Gitlab (might need another access token with the scope `read_registry`):
>
> `docker login registry.gitlab.cc-asp.fraunhofer.de/ontotrans`
>
> And then bring up the container with:
>
> `docker-compose up -d`
>
> Then connect this notebook to the `ipkernel` of the `oteapi`-container in order to run the example
> 

Now, install python-dotenv in order to read the access token and the needed plugins for the oteapi:

In [1]:
!pip install otelib==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.8/182.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.6/405.6 kB 10.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 10.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 12.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 8.0 MB/s e

wrapper and the requirements for its service is installed:

In [2]:
from otelib import OTEClient
import os
import requests
from pprint import pprint

# Configuration
OTEAPI_service_URL = "http://ote_api:8080"
OTEAPI_prefix = "/api/v1"

# Set environment variables
os.environ["OTEAPI_prefix"] = OTEAPI_prefix

client = OTEClient(OTEAPI_service_URL)

In [3]:
BASE_URL = "http://simphony_server:8080"

In [4]:
ontologypath = f"{BASE_URL}/ontology"
cudspath = f"{BASE_URL}/examples/simphony/turtle"
source = client.create_dataresource(
    downloadUrl=cudspath,
    mediaType="application/CUDS",
    configuration={
        "ontologyUrl": ontologypath,
        },
)

In [5]:
input_cuds_path = f"{BASE_URL}/examples/simphony/json"
source = client.create_dataresource(
    downloadUrl=input_cuds_path,
    mediaType="application/json",
)

b'{"content":{"@graph":[{"@id":"cuds:bc34f079-3f43-4fc5-a189-e5e71183d54d","@type":"mods:DataPointItem","mods:name":"var5","mods:value":"1.7","mods:isPartOf":{"@id":"cuds:0ef5b540-083c-4d88-82b0-f0ad6cb57a13"}},{"@id":"cuds:3fe5a253-5227-43a7-9124-348fe3db044b","@type":"mods:DataPointItem","mods:value":"0.1","mods:name":"var4","mods:isPartOf":{"@id":"cuds:ede09711-5318-4eba-93c3-be6443b1087b"}},{"@id":"cuds:f2b16624-ef7c-414d-9212-c772f151f130","@type":"mods:Variable","mods:maximum":"None","mods:type":"output","mods:objective":"None","mods:name":"var6","mods:minimum":"None","mods:weight":"None","mods:isPartOf":{"@id":"cuds:31fd7933-2e3e-4b00-a175-fe2ad47d46e5"}},{"@id":"cuds:777c5c44-2ca0-463d-a122-a1fcfcc6c3b8","@type":"mods:DataPointItem","mods:value":"1.2","mods:name":"var5","mods:isPartOf":{"@id":"cuds:ede09711-5318-4eba-93c3-be6443b1087b"}},{"@id":"cuds:5a2f3404-13b4-4d6c-83b5-e940f6b608ac","@type":"mods:DataPointItem","mods:name":"var2","mods:value":"0.8","mods:isPartOf":{"@id":"

In [10]:
output_cuds_path = "/tmp/oteapi/ontology/outputCUDS.ttl"
transformation = client.create_transformation(**{
        "transformationType": "application/simulation",
        "configuration": {
            "timeout": 10,
            "backend": "celery",
            "run": {
                "request_input":{
                    "url": f"{BASE_URL}/run",
                    "method":"post",
                    "content_type":"application/ld+json",
                    "payload_from_session_cache":"key"
                },
                "request_output": {
                    "content_type":"application/json",
                    "payload_to_session": {"id": "$.task_id"}
                }
            },
            "status": {
                "request_input":{
                    "url": f"{BASE_URL}/status/<task_id>",
                    "method": "get",
                    "arg_from_session": {
                        "id": "<task_id>"
                    }
                },
                "request_output":{
                    "content_type":"application/json",
                    "payload_to_session": {
                        "status": "$.status",
                        "message": "$.message"
                    }
                }
            },
            "results": {
                "request_input": {
                    "url": f"{BASE_URL}/get/<task_id>",
                    "arg_from_session": {
                        "id": "<task_id>"
                    },
                    "method": "get",
                },
                "request_output": {
                    "content_type":"text/turtle; charset=utf-8",
                    "payload_to_cache": "output_cuds_key",
                }
            },
            "upload": {
                "functionType": "function/SaveGraph",
                "configuration": {
                    "fileDestination": output_cuds_path,
                    "graph_sessionKey": "output_cuds_key",
                }
            }
        },
        "status_mapping": {
            "RUNNING": "RUNNING",
            "PENDING": "PENDING",
            "FAILURE": "FAILURE",
            "SUCCESS": "SUCCESS"
        },
    }
)

In [11]:
pipeline = source >> transformation
pipeline.get()

b'{"output_cuds_key":"e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855","content":{"@graph":[{"@id":"cuds:bc34f079-3f43-4fc5-a189-e5e71183d54d","@type":"mods:DataPointItem","mods:name":"var5","mods:value":"1.7","mods:isPartOf":{"@id":"cuds:0ef5b540-083c-4d88-82b0-f0ad6cb57a13"}},{"@id":"cuds:3fe5a253-5227-43a7-9124-348fe3db044b","@type":"mods:DataPointItem","mods:value":"0.1","mods:name":"var4","mods:isPartOf":{"@id":"cuds:ede09711-5318-4eba-93c3-be6443b1087b"}},{"@id":"cuds:f2b16624-ef7c-414d-9212-c772f151f130","@type":"mods:Variable","mods:maximum":"None","mods:type":"output","mods:objective":"None","mods:name":"var6","mods:minimum":"None","mods:weight":"None","mods:isPartOf":{"@id":"cuds:31fd7933-2e3e-4b00-a175-fe2ad47d46e5"}},{"@id":"cuds:777c5c44-2ca0-463d-a122-a1fcfcc6c3b8","@type":"mods:DataPointItem","mods:value":"1.2","mods:name":"var5","mods:isPartOf":{"@id":"cuds:ede09711-5318-4eba-93c3-be6443b1087b"}},{"@id":"cuds:5a2f3404-13b4-4d6c-83b5-e940f6b608ac","@type